In [2]:
from common import openai_api_key

from langchain import OpenAI
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import os
import streamlit as st

In [4]:
os.environ["OPENAI_API_KEY"]=openai_api_key

In [5]:
current_directory = os.getcwd()
print(current_directory)

/Users/bhargobdeka/Desktop/Projects/academic-paper-summarizer/experiments


In [19]:

pdfreader = PdfReader('../docs/housing.pdf')
num_pages = len(pdfreader.pages)
print(f'Total number of pages: {num_pages}')

Total number of pages: 48


In [20]:
text = pdfreader.pages[0].extract_text()
text = text.replace('\n', ' ') # remove new line characters
text = text.replace('\x0c', ' ') # remove the form feed character

text = ' '.join(text.split()) # replace multiple spaces with single space
print(text)

HOUSING MARKET INFORMATION Housing Supply Report Subscribe cmhc.ca/researchnewsletterCANADIAN METROPOLITAN AREAS October 2023


In [ ]:
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

print(raw_text)
      

In [24]:
# We need to split the text using Character Text Split such that it should not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)

In [27]:
texts = text_splitter.split_text(raw_text) # split the text into chunks
len(texts)

161

In [28]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [30]:
document_search = FAISS.from_texts(texts, embeddings)
document_search

In [32]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [33]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [34]:
query = "What are the main highlights for Montreal?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The main highlights for Montreal are that total housing starts declined significantly in the first half of 2023, driven largely by declines in condominium apartments and purpose-built rental apartments. This decline was more reflective of the recent deterioration in financial conditions, as small and low-rise apartment structures in Montreal take less time to plan and build. The high share of total starts accounted for by rental apartments likely also contributed to Montréal’s faster response to higher financing costs.'